In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sim_functions import *

In [16]:
sccm_total = 15300
T_C = 650
T_STD = 298
p = 100 #mbar
p_atm = 1013
flow_T = sccm_total * ((T_C + T_STD)/ T_STD) * p_atm/p # cm3/60s
print("cm^3/min = ", flow_T)

cross_section = 13*4.5 #cm
flow_m_s = flow_T/cross_section/60 /100 #m/s
print("flow speed m/s = ", flow_m_s)

Diameter_hydraulic = 2*0.13*0.045/(0.13+0.045)
M = 2.016 *1e-3 #kg/mol
rho = (p*100) * M /8.31 /(T_C+T_STD)
mu = 0.192 * 1e-3 * 1e-1 # g/cm/s to Pa/s #https://pmc.ncbi.nlm.nih.gov/articles/PMC6728481/ # Dynamic viscosity
Re = rho * Diameter_hydraulic * flow_m_s / mu
print("Re = ", Re)

cm^3/min =  493052.25503355707
flow speed m/s =  1.4047072792978832
Re =  12.517384246359603


In [ ]:
D =

Pr = mu /rho /